In [18]:
import folium
import random
import math
import pandas as pd

In [19]:
# Load AWS Cloud Regions Dataset
dataset_path = "aws_cloud_locations.csv"  # Update with the correct path
df = pd.read_csv(dataset_path)

In [20]:
df

,Code,City,Country,Lat,Long
0,us-east-2,"Hillard, OH",USA,40.062924,-83.134584
1,us-east-1,"Sterling, VA",USA,39.021184,-77.440525
2,us-west-1,"Santa Clara, CA",USA,37.390164,-121.968163
3,us-west-2,"Boardman, OR",USA,45.845729,-119.671374
4,af-south-1,Cape Town,South Africa,-33.972087,18.411810
5,ap-east-1,Hong Kong,Hong Kong,22.352674,113.987616
6,ap-south-1,Mumbai,India,19.082198,72.741102
7,ap-northeast-3,Osaka,Japan,34.677623,135.416025
8,ap-northeast-2,Seoul,South Korea,37.565017,126.849467
9,ap-southeast-1,Singapore,Singapore,1.343742,103.683960


In [21]:
# Extract cloud region locations (latitude and longitude)
cloud_regions = list(zip(df["Lat"], df["Long"]))  # Convert to a list of tuples

In [22]:
# Assign random capacities and operational costs for the regions
region_capacities = [random.randint(300, 600) for _ in range(len(cloud_regions))]
region_operational_costs = [random.randint(50, 150) for _ in range(len(cloud_regions))]

In [23]:
# Randomly generate user locations worldwide (latitude, longitude)
NUM_USERS = 50
users = [(random.uniform(-60, 60), random.uniform(-180, 180)) for _ in range(NUM_USERS)]

In [24]:
# Parameters for Simulated Annealing
INITIAL_TEMP = 1000
FINAL_TEMP = 1
ALPHA = 0.95
MAX_ITER = 100

In [25]:
# Cost function
def calculate_advanced_cost(region_locations, assignments):
    total_cost = 0
    region_loads = [0] * len(region_locations)

    for user_idx, region_idx in enumerate(assignments):
        latency = haversine_distance(users[user_idx], region_locations[region_idx])  # Latency as distance
        total_cost += latency  # Add latency as cost
        region_loads[region_idx] += 1  # Increment load on the assigned region

    # Add operational costs and overcapacity penalties
    for region_idx, load in enumerate(region_loads):
        total_cost += region_operational_costs[region_idx]  # Fixed operational cost
        if load > region_capacities[region_idx]:  # Overcapacity penalty
            total_cost += (load - region_capacities[region_idx]) * 50  # Penalty per extra user

    return total_cost

In [26]:
# Haversine distance for real-world coordinates
def haversine_distance(coord1, coord2):
    R = 6371  # Earth radius in km
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)

    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

In [27]:
# Generate a random initial solution
def initialize_solution():
    assignments = [random.randint(0, len(cloud_regions) - 1) for _ in range(NUM_USERS)]
    return assignments

In [28]:
# Generate a neighboring solution
def generate_neighbor(current_solution):
    neighbor = current_solution[:]
    user_to_reassign = random.randint(0, NUM_USERS - 1)
    neighbor[user_to_reassign] = random.randint(0, len(cloud_regions) - 1)
    return neighbor

In [29]:
# Visualize the solution on a real map
def plot_solution_on_map(users, cloud_regions, solution):
    map_center = (20, 0)  # Approximate center of the world
    cloud_map = folium.Map(location=map_center, zoom_start=2)

    # Add cloud regions to the map
    for i, (lat, lon) in enumerate(cloud_regions):
        folium.Marker(location=(lat, lon), popup=f"Cloud Region {i}", icon=folium.Icon(color="red")).add_to(cloud_map)

    # Add users to the map
    for i, (lat, lon) in enumerate(users):
        folium.Marker(location=(lat, lon), popup=f"User {i}", icon=folium.Icon(color="blue")).add_to(cloud_map)

        # Draw lines from users to assigned cloud regions
        region_idx = solution[i]
        folium.PolyLine([(lat, lon), cloud_regions[region_idx]], color="gray").add_to(cloud_map)

    return cloud_map

In [30]:
# Simulated Annealing Algorithm
def simulated_annealing(users, cloud_regions):
    current_solution = initialize_solution()
    current_cost = calculate_advanced_cost(cloud_regions, current_solution)
    best_solution = current_solution
    best_cost = current_cost

    temperature = INITIAL_TEMP

    while temperature > FINAL_TEMP:
        for _ in range(MAX_ITER):
            neighbor_solution = generate_neighbor(current_solution)
            neighbor_cost = calculate_advanced_cost(cloud_regions, neighbor_solution)

            delta = neighbor_cost - current_cost
            if neighbor_cost < current_cost or random.random() < math.exp(-delta / temperature):
                current_solution = neighbor_solution
                current_cost = neighbor_cost

            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost

        temperature *= ALPHA

    return best_solution, best_cost

In [31]:
best_solution, best_cost = simulated_annealing(users, cloud_regions)

# Print results
print("Best cost:", best_cost)
print("User assignments to cloud regions:", best_solution)

Best cost: 153924.78812804748
User assignments to cloud regions: [19, 9, 10, 16, 16, 20, 10, 20, 10, 9, 2, 4, 20, 1, 6, 4, 4, 20, 20, 20, 10, 9, 1, 12, 3, 6, 4, 4, 11, 9, 19, 11, 11, 20, 4, 1, 9, 20, 20, 3, 1, 20, 11, 10, 20, 18, 4, 2, 20, 16]


In [32]:
# Generate and show the map
cloud_map = plot_solution_on_map(users, cloud_regions, best_solution)
cloud_map.save("cloud_allocation_map.html")
print("Map saved as 'cloud_allocation_map.html'. Open it in a browser.")

Map saved as 'cloud_allocation_map.html'. Open it in a browser.
